<a href="https://colab.research.google.com/github/ndb796/Deep-Learning-Paper-Review-and-Practice/blob/master/code_practices/ResNet18_CIFAR10_Train.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch

torch.cuda.is_available()

True

In [2]:
import torch
import random
import torch.backends.cudnn as cudnn

torch.manual_seed(0)
torch.cuda.manual_seed(0)
torch.cuda.manual_seed_all(0)
cudnn.benchmark = False
cudnn.deterministic = True
random.seed(0)

#### ResNet18 모델 정의 및 인스턴스 초기화

In [3]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.backends.cudnn as cudnn
import torch.optim as optim
import os


# ResNet18을 위해 최대한 간단히 수정한 BasicBlock 클래스 정의
class BasicBlock(nn.Module):
    def __init__(self, in_planes, planes, stride=1):
        super(BasicBlock, self).__init__()

        # 3x3 필터를 사용 (너비와 높이를 줄일 때는 stride 값 조절)
        self.conv1 = nn.Conv2d(in_planes, planes, kernel_size=3, stride=stride, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(planes) # 배치 정규화(batch normalization)

        # 3x3 필터를 사용 (패딩을 1만큼 주기 때문에 너비와 높이가 동일)
        self.conv2 = nn.Conv2d(planes, planes, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(planes) # 배치 정규화(batch normalization)

        self.shortcut = nn.Sequential() # identity인 경우
        if stride != 1: # stride가 1이 아니라면, Identity mapping이 아닌 경우
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_planes, planes, kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(planes)
            )

    def forward(self, x):
        out = F.relu(self.bn1(self.conv1(x)))
        out = self.bn2(self.conv2(out))
        out += self.shortcut(x) # (핵심) skip connection
        out = F.relu(out)
        return out


# ResNet 클래스 정의
class ResNet(nn.Module):
    def __init__(self, block, num_blocks, num_classes=10):
        super(ResNet, self).__init__()
        self.in_planes = 64

        # 64개의 3x3 필터(filter)를 사용
        self.conv1 = nn.Conv2d(3, 64, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(64)
        self.layer1 = self._make_layer(block, 64, num_blocks[0], stride=1)
        self.layer2 = self._make_layer(block, 128, num_blocks[1], stride=2)
        self.layer3 = self._make_layer(block, 256, num_blocks[2], stride=2)
        self.layer4 = self._make_layer(block, 512, num_blocks[3], stride=2)
        self.linear = nn.Linear(512, num_classes)

    def _make_layer(self, block, planes, num_blocks, stride):
        strides = [stride] + [1] * (num_blocks - 1)
        layers = []
        for stride in strides:
            layers.append(block(self.in_planes, planes, stride))
            self.in_planes = planes # 다음 레이어를 위해 채널 수 변경
        return nn.Sequential(*layers)

    def forward(self, x):
        out = F.relu(self.bn1(self.conv1(x)))
        out = self.layer1(out)
        out = self.layer2(out)
        out = self.layer3(out)
        out = self.layer4(out)
        out = F.avg_pool2d(out, 4)
        out = out.view(out.size(0), -1)
        out = self.linear(out)
        return out


# ResNet18 함수 정의
def ResNet18():
    return ResNet(BasicBlock, [2, 2, 2, 2])

#### 데이터셋(Dataset) 다운로드 및 불러오기

In [4]:
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import ConcatDataset


data_transformations = {
    "crop_flip": transforms.Compose([  # 크롭 + 뒤집기
    transforms.RandomCrop(32, padding=4),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),]), 

    "color_blur": transforms.Compose([  # 색깔 + 블러
     transforms.ColorJitter(brightness=(0.5, 0.9), 
                           contrast=(0.4, 0.8), 
                           saturation=(0.7, 0.9),
                           hue=(-0.2, 0.2),
                          ),
    transforms.GaussianBlur(kernel_size=(19, 19), sigma=(1.0, 2.0)),
    transforms.ToTensor()]),

    "basic": transforms.Compose([
    transforms.ToTensor()])
}

train_dataset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=data_transformations["basic"])
train_aug_dataset = torchvision.datasets.CIFAR10(root='./data', train=True, transform=data_transformations["crop_flip"])
train_aug_dataset2 = torchvision.datasets.CIFAR10(root='./data', train=True, transform=data_transformations["color_blur"])
                                                  
augmented_train_data = ConcatDataset((train_dataset,train_aug_dataset, train_aug_dataset2 ))

test_dataset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=data_transformations["basic"])

train_loader = torch.utils.data.DataLoader(augmented_train_data, batch_size=128, shuffle=True, num_workers=4)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=100, shuffle=False, num_workers=4)

Files already downloaded and verified
Files already downloaded and verified


#### 환경 설정 및 학습(Training) 함수 정의

In [5]:
device = 'cuda'

net = ResNet18()
net = net.to(device)
net = torch.nn.DataParallel(net)
cudnn.benchmark = True

learning_rate = 0.1
file_name = 'resnet18_cifar10.pt'

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=learning_rate, momentum=0.9, weight_decay=0.0002)


def train(epoch):
    print('\n[ Train epoch: %d ]' % epoch)
    net.train()
    train_loss = 0
    correct = 0
    total = 0
    for batch_idx, (inputs, targets) in enumerate(train_loader):
        inputs, targets = inputs.to(device), targets.to(device)
        optimizer.zero_grad()

        benign_outputs = net(inputs)
        loss = criterion(benign_outputs, targets)
        loss.backward()

        optimizer.step()
        train_loss += loss.item()
        _, predicted = benign_outputs.max(1)

        total += targets.size(0)
        correct += predicted.eq(targets).sum().item()
        
        if batch_idx % 100 == 0:
            print('\nCurrent batch:', str(batch_idx))
            print('Current benign train accuracy:', str(predicted.eq(targets).sum().item() / targets.size(0)))
            print('Current benign train loss:', loss.item())

    print('\nTotal benign train accuarcy:', 100. * correct / total)
    print('Total benign train loss:', train_loss)


def test(epoch):
    print('\n[ Test epoch: %d ]' % epoch)
    net.eval()
    loss = 0
    correct = 0
    total = 0

    for batch_idx, (inputs, targets) in enumerate(test_loader):
        inputs, targets = inputs.to(device), targets.to(device)
        total += targets.size(0)

        outputs = net(inputs)
        loss += criterion(outputs, targets).item()

        _, predicted = outputs.max(1)
        correct += predicted.eq(targets).sum().item()

    print('\nTest accuarcy:', 100. * correct / total)
    print('Test average loss:', loss / total)

    state = {
        'net': net.state_dict()
    }
    if not os.path.isdir('checkpoint'):
        os.mkdir('checkpoint')
    torch.save(state, './checkpoint/' + file_name)
    print('Model Saved!')


def adjust_learning_rate(optimizer, epoch):
    lr = learning_rate
    if epoch >= 50:
        lr = 0.01
    if epoch >= 75:
        lr = 0.001
    for param_group in optimizer.param_groups:
        param_group['lr'] = lr

#### 학습(Training) 진행

* 대략 20번의 epoch 이후에도 85%가량의 test accuracy를 얻을 수 있습니다.

In [6]:
# for epoch in range(0, 200):
for epoch in range(0, 100):
    adjust_learning_rate(optimizer, epoch)
    train(epoch)
    test(epoch)


[ Train epoch: 0 ]

Current batch: 0
Current benign train accuracy: 0.0703125
Current benign train loss: 2.3962252140045166

Current batch: 100
Current benign train accuracy: 0.1796875
Current benign train loss: 2.0725607872009277

Current batch: 200
Current benign train accuracy: 0.328125
Current benign train loss: 1.9170618057250977

Current batch: 300
Current benign train accuracy: 0.2890625
Current benign train loss: 1.884389042854309

Current batch: 400
Current benign train accuracy: 0.359375
Current benign train loss: 1.8013765811920166

Current batch: 500
Current benign train accuracy: 0.2734375
Current benign train loss: 1.9485540390014648

Current batch: 600
Current benign train accuracy: 0.328125
Current benign train loss: 1.747280240058899

Current batch: 700
Current benign train accuracy: 0.3671875
Current benign train loss: 1.6286120414733887

Current batch: 800
Current benign train accuracy: 0.40625
Current benign train loss: 1.6790680885314941

Current batch: 900
Curren


Current batch: 800
Current benign train accuracy: 0.8125
Current benign train loss: 0.4613684117794037

Current batch: 900
Current benign train accuracy: 0.8828125
Current benign train loss: 0.28971922397613525

Current batch: 1000
Current benign train accuracy: 0.875
Current benign train loss: 0.41113272309303284

Current batch: 1100
Current benign train accuracy: 0.875
Current benign train loss: 0.3577149510383606

Total benign train accuarcy: 84.868
Total benign train loss: 511.1780013144016

[ Test epoch: 5 ]

Test accuarcy: 80.02
Test average loss: 0.006151851558685303
Model Saved!

[ Train epoch: 6 ]

Current batch: 0
Current benign train accuracy: 0.859375
Current benign train loss: 0.3509584367275238

Current batch: 100
Current benign train accuracy: 0.890625
Current benign train loss: 0.2990003526210785

Current batch: 200
Current benign train accuracy: 0.8671875
Current benign train loss: 0.4469633400440216

Current batch: 300
Current benign train accuracy: 0.8359375
Current


Current batch: 200
Current benign train accuracy: 0.921875
Current benign train loss: 0.290919691324234

Current batch: 300
Current benign train accuracy: 0.859375
Current benign train loss: 0.4103128910064697

Current batch: 400
Current benign train accuracy: 0.9140625
Current benign train loss: 0.30235299468040466

Current batch: 500
Current benign train accuracy: 0.890625
Current benign train loss: 0.30305105447769165

Current batch: 600
Current benign train accuracy: 0.8984375
Current benign train loss: 0.3229539692401886

Current batch: 700
Current benign train accuracy: 0.8828125
Current benign train loss: 0.2953115999698639

Current batch: 800
Current benign train accuracy: 0.890625
Current benign train loss: 0.31113794445991516

Current batch: 900
Current benign train accuracy: 0.921875
Current benign train loss: 0.29451805353164673

Current batch: 1000
Current benign train accuracy: 0.90625
Current benign train loss: 0.25623732805252075

Current batch: 1100
Current benign tra


Current batch: 1000
Current benign train accuracy: 0.859375
Current benign train loss: 0.29631727933883667

Current batch: 1100
Current benign train accuracy: 0.890625
Current benign train loss: 0.251781702041626

Total benign train accuarcy: 90.96466666666667
Total benign train loss: 308.2171124815941

[ Test epoch: 16 ]

Test accuarcy: 84.66
Test average loss: 0.004746093686670065
Model Saved!

[ Train epoch: 17 ]

Current batch: 0
Current benign train accuracy: 0.90625
Current benign train loss: 0.2531924545764923

Current batch: 100
Current benign train accuracy: 0.9140625
Current benign train loss: 0.2313482165336609

Current batch: 200
Current benign train accuracy: 0.9140625
Current benign train loss: 0.2147125005722046

Current batch: 300
Current benign train accuracy: 0.90625
Current benign train loss: 0.2534984052181244

Current batch: 400
Current benign train accuracy: 0.921875
Current benign train loss: 0.28621721267700195

Current batch: 500
Current benign train accuracy:


Current batch: 400
Current benign train accuracy: 0.9453125
Current benign train loss: 0.15219098329544067

Current batch: 500
Current benign train accuracy: 0.90625
Current benign train loss: 0.2508106827735901

Current batch: 600
Current benign train accuracy: 0.8515625
Current benign train loss: 0.28829097747802734

Current batch: 700
Current benign train accuracy: 0.921875
Current benign train loss: 0.20055930316448212

Current batch: 800
Current benign train accuracy: 0.9296875
Current benign train loss: 0.2453056275844574

Current batch: 900
Current benign train accuracy: 0.9375
Current benign train loss: 0.2650649845600128

Current batch: 1000
Current benign train accuracy: 0.890625
Current benign train loss: 0.32487910985946655

Current batch: 1100
Current benign train accuracy: 0.9140625
Current benign train loss: 0.25687161087989807

Total benign train accuarcy: 91.632
Total benign train loss: 287.9972823187709

[ Test epoch: 22 ]

Test accuarcy: 85.48
Test average loss: 0.0

...


Current batch: 700
Current benign train accuracy: 0.9921875
Current benign train loss: 0.028355780988931656

Current batch: 800
Current benign train accuracy: 1.0
Current benign train loss: 0.010362918488681316

Current batch: 900
Current benign train accuracy: 1.0
Current benign train loss: 0.005383627489209175

Current batch: 1000
Current benign train accuracy: 0.9921875
Current benign train loss: 0.038801856338977814

Current batch: 1100
Current benign train accuracy: 0.9921875
Current benign train loss: 0.02783903293311596

Total benign train accuarcy: 99.55466666666666
Total benign train loss: 17.00901671143947

[ Test epoch: 72 ]

Test accuarcy: 92.43
Test average loss: 0.0029373904425650833
Model Saved!

[ Train epoch: 73 ]

Current batch: 0
Current benign train accuracy: 0.9921875
Current benign train loss: 0.021965017542243004

Current batch: 100
Current benign train accuracy: 0.9921875
Current benign train loss: 0.013792878948152065

Current batch: 200
Current benign train a


Current batch: 200
Current benign train accuracy: 1.0
Current benign train loss: 0.0012904792092740536

Current batch: 300
Current benign train accuracy: 1.0
Current benign train loss: 0.005942390765994787

Current batch: 400
Current benign train accuracy: 1.0
Current benign train loss: 0.0039968048222362995

Current batch: 500
Current benign train accuracy: 0.9921875
Current benign train loss: 0.02110307291150093

Current batch: 600
Current benign train accuracy: 1.0
Current benign train loss: 0.006152712740004063

Current batch: 700
Current benign train accuracy: 0.984375
Current benign train loss: 0.023277483880519867

Current batch: 800
Current benign train accuracy: 1.0
Current benign train loss: 0.0027044694870710373

Current batch: 900
Current benign train accuracy: 1.0
Current benign train loss: 0.003018076065927744

Current batch: 1000
Current benign train accuracy: 1.0
Current benign train loss: 0.0037078398745507

Current batch: 1100
Current benign train accuracy: 0.9921875


Current batch: 1100
Current benign train accuracy: 1.0
Current benign train loss: 0.001385648618452251

Total benign train accuarcy: 99.91
Total benign train loss: 5.836065023962874

[ Test epoch: 83 ]

Test accuarcy: 93.12
Test average loss: 0.002553017885610461
Model Saved!

[ Train epoch: 84 ]

Current batch: 0
Current benign train accuracy: 1.0
Current benign train loss: 0.006529269274324179

Current batch: 100
Current benign train accuracy: 1.0
Current benign train loss: 0.0011866528075188398

Current batch: 200
Current benign train accuracy: 1.0
Current benign train loss: 0.0015717000933364034

Current batch: 300
Current benign train accuracy: 0.9921875
Current benign train loss: 0.012074720114469528

Current batch: 400
Current benign train accuracy: 1.0
Current benign train loss: 0.0008623884059488773

Current batch: 500
Current benign train accuracy: 1.0
Current benign train loss: 0.003314587054774165

Current batch: 600
Current benign train accuracy: 1.0
Current benign train 


Current batch: 700
Current benign train accuracy: 1.0
Current benign train loss: 0.0023650662042200565

Current batch: 800
Current benign train accuracy: 1.0
Current benign train loss: 0.0017371969297528267

Current batch: 900
Current benign train accuracy: 1.0
Current benign train loss: 0.0012789920438081026

Current batch: 1000
Current benign train accuracy: 1.0
Current benign train loss: 0.004606534726917744

Current batch: 1100
Current benign train accuracy: 1.0
Current benign train loss: 0.0008379807695746422

Total benign train accuarcy: 99.92266666666667
Total benign train loss: 4.654111712850863

[ Test epoch: 89 ]

Test accuarcy: 93.08
Test average loss: 0.0025156381249427794
Model Saved!

[ Train epoch: 90 ]

Current batch: 0
Current benign train accuracy: 1.0
Current benign train loss: 0.006895905360579491

Current batch: 100
Current benign train accuracy: 1.0
Current benign train loss: 0.002533411141484976

Current batch: 200
Current benign train accuracy: 1.0
Current beni


Current batch: 200
Current benign train accuracy: 1.0
Current benign train loss: 0.0034538148902356625

Current batch: 300
Current benign train accuracy: 1.0
Current benign train loss: 0.002244453877210617

Current batch: 400
Current benign train accuracy: 1.0
Current benign train loss: 0.00077015720307827

Current batch: 500
Current benign train accuracy: 1.0
Current benign train loss: 0.00142535415943712

Current batch: 600
Current benign train accuracy: 1.0
Current benign train loss: 0.004285948351025581

Current batch: 700
Current benign train accuracy: 1.0
Current benign train loss: 0.000927022541873157

Current batch: 800
Current benign train accuracy: 1.0
Current benign train loss: 0.0022091339342296124

Current batch: 900
Current benign train accuracy: 1.0
Current benign train loss: 0.0040138582699000835

Current batch: 1000
Current benign train accuracy: 1.0
Current benign train loss: 0.0007099843351170421

Current batch: 1100
Current benign train accuracy: 1.0
Current benign